In [ ]:
import jax.numpy as jnp
import numpy as np

import pymudokon as pm

import matplotlib.pyplot as plt

In [ ]:
particles_per_cell = 4

total_steps, output_steps, output_start = 3000, 100, 0

circle1_center = np.array([0.255, 0.255])
circle2_center = np.array([0.745, 0.745])
circle_radius = 0.2


In [ ]:
def create_circle(center: np.array, radius: float, cell_size: float, ppc: int = 2):
    """Generate a circle of particles.

    Args:
        center (np.array): center of the circle
        radius (float): radius of the circle
        cell_size (float): size of the background grid cells
        ppc (int, optional): particles per cell. Defaults to 2.

    Returns:
        np.array: coordinates of the particles
    """
    start, end = center - radius, center + radius
    spacing = cell_size / (ppc / 2)
    tol = +0.00005  # Add a tolerance to avoid numerical issues
    x = np.arange(start[0], end[0] + spacing, spacing) + 0.5 * spacing
    y = np.arange(start[1], end[1] + spacing, spacing) + 0.5 * spacing
    xv, yv = np.meshgrid(x, y)
    grid_coords = np.array(list(zip(xv.flatten(), yv.flatten()))).astype(np.float64)
    circle_mask = (grid_coords[:, 0] - center[0]) ** 2 + (grid_coords[:, 1] - center[1]) ** 2 < radius**2 + tol
    return grid_coords[circle_mask]

In [ ]:
# Create circles of particles and concatenate them into a single array
circle_centers = np.array([circle1_center, circle2_center])
cell_size = 0.05
particles_per_cell = 2


circles = [create_circle(center, circle_radius, cell_size, particles_per_cell) for center in circle_centers]
pos = np.vstack(circles)


velocities = [np.full(circle.shape, 0.1 if i == 0 else -0.1) for i, circle in enumerate(circles)]
vels = np.vstack(velocities)


In [ ]:
plt.scatter(pos[:,0], pos[:,1]).axes.set(xlim=(0, 1), ylim=(0, 1))

In [ ]:
particles = pm.Particles.register(positions=jnp.array(pos), velocities=jnp.array(vels), original_density=1000)
particles = particles.calculate_volume(cell_size, particles_per_cell=4)
particles = particles.replace(masses=1000 * particles.volumes)
nodes = pm.Nodes.register(origin=jnp.array([0.0, 0.0]), end=jnp.array([1.0, 1.0]), node_spacing=cell_size, particles_per_cell=4)

material = pm.LinearIsotropicElastic.register(E=1000.0, nu=0.3, num_particles=len(pos), dim=2)

shapefunctions = pm.LinearShapeFunction.register(len(pos), 2)

In [ ]:
usl = pm.USL.register(
    particles=particles, nodes=nodes, materials=[material], shapefunctions=shapefunctions, alpha=1.0, dt=0.001
)

In [ ]:
import pyvista as pv
# requires 
# apt-get install ffmpeg libsm6 libxext6  -y


plotter = pv.Plotter(notebook=True)
points_3d = jnp.pad(pos, [(0, 0), (0, 1)], mode="constant").__array__()

cloud = pv.PolyData(points_3d)

plotter.add_mesh(
    points_3d,
     color="red", point_size=10
    # scalars=z.ravel(),
    # lighting=False,
    # show_edges=True,
    # scalar_bar_args={"title": "Height"},
    # clim=[-1, 1],
)

plotter.show(jupyter_backend='trame')
# plotter.open_gif("impact.gif")

# plotter.write_frame()

# def some_callback(package):

#     usl, step = package  # unused intentionally
    
#     points = usl.particles.positions


#     points_3d = jnp.pad(points, [(0, 0), (0, 1)], mode="constant").__array__()


#     plotter.update_coordinates(points_3d, render=False)
#     plotter.write_frame()
    
    
# #     velocities_3d = jnp.pad(velocities, [(0, 0), (0, 1)], mode="constant").__array__()

#     print(f"[JAX] output {step}/{total_steps} \n")

# #     cloud = pv.PolyData(points_3d)

# #     # # Add velocities as point data
# #     cloud.point_data["velocities"] = velocities_3d

# #     cloud.save(f"./output/particles{step}.vtp")


# usl = usl.solve(num_steps=total_steps, output_step=output_steps, output_function=some_callback)

# plotter.close()

In [1]:
import pyvista as pv
sphere = pv.Sphere()

# short example
sphere.plot(jupyter_backend='trame')

# long example
plotter = pv.Plotter(notebook=True)
plotter.add_mesh(sphere)
plotter.show(jupyter_backend='trame')